In [ ]:
import os
import json
import numpy as np
import torch

# Meta Info

In [ ]:
base_grasps_path = '/home/felipe/Downloads/fullrobots-sharp_lift_numg_64/'

result_folder = "/home/felipe/Documents/GenDexGrasp/results/ablations_fullrobots_numg64_penw60"

results_file = "sh12.json"

results_path = os.path.join(result_folder, results_file)

with open(results_path, "r") as jf:
    results = json.load(jf)

object_list = results.keys()
print(object_list)

In [ ]:
gripper_name = None
if 'bh' in results_file:
    gripper_name = 'barrett'
elif 'ez' in results_file:
    gripper_name = 'ezgripper'
elif 'sh' in results_file:
    gripper_name = 'shadowhand'

print("Computing metrics for gripper:", gripper_name)

grasps_path = os.path.join(base_grasps_path, "ood", gripper_name)
print("Grasps Path (Dir for per-object grasp opt results):\n", grasps_path)

# Success Rate

In [ ]:
total_grasps = 0
total_success = 0
for obj in object_list:
    #print(obj)
    if obj == 'cfg' or obj =='adam_cfg':
        continue
    try:
        tmp_n = results[obj]['total_num'] 
        tmp_s = results[obj]['succ_num']
        print(f"Object {obj} success_rate = ", round((tmp_s/tmp_n),2))
        total_grasps+=tmp_n
        total_success+=tmp_s
    except:
        print(f"WARNING: Object {obj} result values were not found")
        
print('\n',f"Total Grasps = {total_grasps}")
print(f" Total Successful Grasps = {total_success}")
print('\n',f"Total Success Rate = {round(total_success/total_grasps,3)}")


# Grasp Diversity

**Note: check path to grasps file below**

In [ ]:
grasps = dict()
data_listdir = os.listdir(grasps_path)
print(data_listdir)

In [ ]:
# Populate grasp info
for file in data_listdir:
    obj_name = file.split('-')[2][:-3]
    if obj_name not in object_list:
        continue
    i_record = torch.load(os.path.join(grasps_path,file))
    grasps[obj_name] = i_record['q_data']

# print(grasps)

In [ ]:
successful_grasps = []
for obj in object_list:
    if obj == 'cfg' or obj =='adam_cfg':
        continue
    flags = np.asarray(results[obj]['succ_flag'])
    tmp_s = np.asarray(grasps[obj].cpu())
    succ_grasps = tmp_s[flags][:,9:] # Remove the translation and orientation
    #print(succ_grasps.shape, len(successful_grasps))
    if len(successful_grasps)==0:
        successful_grasps = succ_grasps
    else:
        successful_grasps = np.concatenate((successful_grasps,succ_grasps),axis =0)
print(successful_grasps[0])

## Diversity Results

In [ ]:
std = np.std(successful_grasps)
std_q = np.std(successful_grasps, axis=0)
print("Grasp Diversity Metric for individual DoFs:", std_q)

# This is incorrect!! We should take RMSE for across all joints
print("Grasp Diversity Metric for all DoFs:", round(std,2))


In [ ]:
# First Get the Variances -> hat(V) -> hat(s)
# Basicall RMSE -- root mean square
print("Mean of std_q:", np.mean(std_q))
print("RMSE of std_q:", np.sqrt(np.mean(std_q**2)))